In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
import re

In [ ]:
# Setting results path
data_path='/content/drive/MyDrive/Dacon/뉴스 토픽 분류 AI/data/'
model_path='/content/drive/MyDrive/Dacon/뉴스 토픽 분류 AI/model/'
sub_path='/content/drive/MyDrive/Dacon/뉴스 토픽 분류 AI/sub/'

# 4. Refinement of back translated data

+ Correct incorrectly translated parts
+ Calibrate sentence styles to resemble training data
+ Special Character Processing

In [ ]:
back_train=pd.read_csv(data_path+'back_train_fin.csv')

# Replace string
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

# During the translation, I found words that were not related to the original sentence were put in.
d = { "[프로젝트 리포트]": "", "[연구보고]": "","[시론]":"",'[특별기획]':"",'[행사참가기]':'','[학술발표회]':'','[논문초록]':'','[발언]':'','[기술정보]':'','[국제학술발표회]':'',
     '[서평]':'','[학회행사]':'','[행사보고]':'','[학술보고]':'','[세미나]':''}

In [ ]:
for i in range(0,len(back_train)):
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('?',' ').replace('!',' ').replace(':',' ').replace(';','').replace('【·】','').replace('】','').replace('=','').replace(' → ','→').replace('→→','')

  back_train.at[i,'back_title']=replace_all(back_train.back_title[i],d)

  back_train.at[i,'back_title']=back_train['back_title'][i].replace('[','').replace(']',' ').replace('(',' ').replace(')',' ').replace('+','＋').replace('\'','').replace("\"","")

  back_train.at[i,'back_title']=re.sub(r'([\/]+)(?![0-9])', r'·', back_train['back_title'][i]).replace('· ','·')

  back_train.at[i,'back_title']=re.sub(r'([#]+[0-9])', r'\1위', back_train['back_title'][i]).replace('#','')

  if (back_train['title'][i].find('·')!=-1) & (back_train['back_title'][i].find('/')!=-1):
    back_train.at[i,'back_title']=back_train['back_title'][i].replace('/','·')

  if (back_train['title'][i].find('·')!=-1) & (back_train['back_title'][i].find('-')!=-1):
    back_train.at[i,'back_title']=back_train['back_title'][i].replace('-','·')

  if (back_train['title'][i].find('∼')!=-1) & (back_train['back_title'][i].find('-')!=-1):
    back_train.at[i,'back_title']=back_train['back_title'][i].replace('-','∼')

  if (back_train['title'][i].find('∼')!=-1) & (back_train['back_title'][i].find('-')!=-1):
    back_train.at[i,'back_title']=back_train['back_title'][i].replace('-','∼')

  if (back_train['title'][i].find('·')!=-1) & (back_train['back_title'][i].find(',')!=-1):
    back_train.at[i,'back_title']=back_train['back_title'][i].replace(',','·')

  back_train.at[i,'back_title']=back_train['back_title'][i].replace('북-미','북미').replace('미-중','미중').replace('원-달러','원달러').replace('미-러','미러').replace('K-리그','K리그').replace('convid-19','코로나19').replace('미-이란','미이란')\
  .replace('한-미','한미').replace('북-중','북중').replace('북-러','북러').replace('북-일','북일').replace('covid-19','코로나19').replace('COVID-19','코로나19').replace('Covid-19','코로나19').replace('페리 세월','세월호').replace('미스터 문','문 대통령')\
  .replace(' ↑','↑').replace('°C','↑').replace('iPhone','아이폰').replace('♬','').replace('ᄌ jeju jeju unique ','').replace('Galaxy Note ','갤노트').replace('Galaxy ','갤럭시').replace('Wi-Fi','와이파이').replace('습니다','다').\
  .replace('입니다','이다').replace('했습니다','했다').replace('나요','나').replace('어요','다').replace('아요','다').replace('까요','까').replace('서요','').replace('해요','')
  
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('    ',' ').replace('   ',' ').replace('  ',' ').replace('· ','·').strip()

In [ ]:
# Make sure have a sentence style that is as similar to the original one as possible
for i in tqdm(range(0,len(back_train))):
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('...','@')
  back_train.at[i,'title']=back_train['title'][i].replace('...','@')

  if (back_train['title'][i].endswith('.')) & ~(back_train['back_title'][i].endswith('.')):
    back_train.at[i,'back_title']=back_train['back_title'][i]+'.'
  if ~(back_train['title'][i].endswith('.')) & (back_train['back_title'][i].endswith('.')):
    back_train.at[i,'back_title']=back_train['back_title'][i][:-1]

  back_train.at[i,'back_title']=back_train['back_title'][i].replace('@','...')
  back_train.at[i,'title']=back_train['title'][i].replace('@','...')

In [ ]:
# Define for special cases and correct them directly
for i in range(0,len(back_train)):
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('. ','…').replace('D…C…','DC').replace('· D…A…L…T…',' ').replace('……','…').strip()

a=back_train[back_train['back_title'].str.contains('….….')].index
b=back_train[back_train['back_title'].str.contains('\.\.…')]['back_title'].index
c=back_train[(back_train['back_title'].str.contains('다\.'))&~(back_train['back_title'].str.contains('\.\.\.'))].index
d=back_train[back_train['back_title'].str.contains('.→.→.')]['back_title'].index
e=back_train[(back_train['back_title'].str.contains('뉴'))&(back_train['title'].str.contains('신간'))].index
f=back_train[(back_train['back_title'].str.contains('있어요'))&~(back_train['back_title'].str.endswith('있어요'))&~(back_train['back_title'].str.contains('있어요…'))&~(back_train['back_title'].str.contains('있어요 '))].index
g=back_train[~(back_train['title'].str.contains('합니다'))&(back_train['back_title'].str.contains('합니다'))].index
h=back_train[(back_train['title'].str.contains('종합'))&((back_train['back_title'].str.endswith('…합성'))|(back_train['back_title'].str.endswith('합성')))].index

for i in a:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('…','')
for i in b:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('..…','…')
for i in c:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('다.','다…')
for i in d:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('→','')
for i in e:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('뉴욕','신간').replace('뉴진',' 신간').replace('뉴','신간 ')  
for i in f:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('있어요','있다…')
for i in g:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('합니다','')
for i in h:
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('…합성','종합').replace('합성','종합')

for i in range(0,len(back_train)):
  back_train.at[i,'back_title']=back_train['back_title'][i].replace('    ',' ').replace('   ',' ').replace('  ',' ').replace(' …','…').replace('… ','…').strip()

In [ ]:
# Final back translated data used in training
back_train.to_csv(data_path+'back_train_fin2.csv',index=False)

back_train.sample(30)

,index,title,topic_idx,back_title
6931,6931,폭우와 함께 마무리되는 아시안게임,5,폭우로 끝나는 아시안게임
8541,8541,남중국해서 美·中 군사적 충돌 가능성 커져…대화 틀 필요,4,남중국해에서 미국과 중국의 군사적 충돌 가능성이 커지고 있다…대화 프레임 필요
7392,7392,트럼프 김정은 더이상 나가도록 내버려둬선 안돼종합2보,4,"트럼프, 김정은 더는 못 가게 해"
38192,38192,전북 주요 대학들 등록금 동결…학부모 부담 덜겠다,2,전북 주요 대학 등록금 동결…부모들의 부담을 덜어줄 거야
14619,14619,kt 지휘봉 서동철 감독 내년 6강 그다음 시즌 정상 도전,5,"서동철 KT 지휘봉 감독 내년 6라운드, 내년 시즌 최고의 도전"
25197,25197,윤석헌 신임 금감원장 8일 취임…삼성증권·삼바 첫 시험대종합,1,윤석헌 신임 금감원장이 8일 취임했다…삼성증권
2960,2960,여자농구 우리은행 신한은행 꺾고 8연승 선두 질주,5,여자농구 우리은행 신한은행이 8연승을 이끌었다
44952,44952,朴대통령 이러려고 대통령 했나하는 자괴감 들 정도속보,6,박 대통령이 이 일로 자괴감을 느낄 정도의 속보였다
43160,43160,게시판 인터파크 고급양장본 한정판매,2,게시판 인터파크의 고급 하드커버 사본 판매 제한
14683,14683,삼성전자 이미지센서란,0,삼성전자 이미지 센서 칼럼
